In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
# normalize the data
train_datagen = ImageDataGenerator(rescale=1./255)

# call the flow from directory method on it to get it to load images from that directory 
# and its sub-directories. name of the sub-direcotries will be the labels for your images 
# that are contained within them. 
train_generator = train_datagen.flow_from_directory(
    train_dir,                    # the directory you're pointing to
    target_size=(300, 300),       # images are resized as they are loaded
    batch_size=128
    class_mode='binary')          #  i.e. it picks between two different things; horses and humans,
# It's a common mistake that people point the generator at the sub-directory. It will fail in that 
# circumstance. You should always point it at the directory that contains sub-directories that 
#contain your images. 

# Now, images might come in all shapes and sizes and unfortunately for training a neural network, 
# the input data all has to be the same size, so the images will need to be resized to make them 
# consistent. The nice thing about this code is that the images are resized for you as 
# they're loaded. So you don't need to preprocess thousands of images on your file system.

# Finally, there's the class mode. Now, this is a binary classifier i.e. it picks between two 

# different things; horses and humans, 

# the validation generator should be exactly the same except of course it points at a different
# directory, the one containing the sub-directories containing the test images.
test_generator = train_datagen.flow_from_directory(
    validation_dir,                   
    target_size=(300, 300),      
    batch_size=128
    class_mode='binary')  

In [4]:
model = tf.keras.models.Sequential([
    # 300x300 size, 3 = RGB colors
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    # one neuron two classes bcs of the different actiavtion function
    # where sigmoid is great for binary classification, where one class will tend 
    # towards zero and the other class tending towards one.
    # You could use two neurons here if you want, and the same softmax function as before, 
    # but for binary this is a bit more efficient.
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

Metal device set to: Apple M1 Pro


2022-03-21 06:06:22.549011: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-21 06:06:22.549156: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

In [6]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',       # binary choice so binary crossentropy
              optimizer=RMSprop(lr=0.001),      # earlier we used an Adam optimizer. Now, you could do that again, 
                                                # but I thought it would be fun to use the RMSprop, where you can adjust the learning rate to experiment with performance. 
              metrics=['acc'])

/Users/tonggihkang/miniforge3/envs/TF/lib/python3.8/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
history = model.fit_generator(     # not model.fit because now we are using a generator instead of datasets
    train_generator,               # This streams the images from the training directory.
    steps_per_epoch=8,             # Remember the batch size you used when you created it, it was 20, 
                                   # that's important in the next step. There are 1,024 images in the training directory, 
                                   # so we're loading them in 128 at a time. So in order to load them all, we need to do 8 batches.
    epochs=15,
    validation_data=validation_generator,
    validation_steps=8,            # It had 256 images, and we wanted to handle them in batches of 32, so we will do 8 steps.
    verbose=2)                     # And the verbose parameter specifies how much to display while training is going on. 

In [ ]:
# PREDICTION
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
    
    # predicting images
    path = '/content/' + fn
    img = image.load_img(path, target_size=(300, 300))  #
    x = image.img_to_array(img)                         # you can load an image and prepare it to 
    x = np.expand_dims(x, axis=0)                       # input into the model with this code. 
    images = np.vstack([x])                             # N.B. match the input dimensions that you 
                                                        # specified when designing the model. 
    
    classes = model.predict(images, batch_size=10)      # You can then call model.predict, passing it 
                                                        # the details, and it will return an array of classes. 
    print(classes[0])
    # In the case of binary classification, this will only contain one item with a 
    # value close to 0 for one class and close to 1 for the other.
    if classes[0]>0.5:
        print(fn + " is a human")
    else:
        print(fn " is a horse")